In [4]:
import hashlib
import random
import simpleNum

In [6]:
def global_print(*names):
    x = lambda s: ["{}", "0x{:x}"] [hasattr(s, 'real')].format(s)
    print("".join("{} = {}\n".format(name, x(globals()[name])) for name in names))

# note: str converts as is, str( [1,2,3,4] ) will convert to "[1,2,3,4]"
def H(*args):  # a one-way hash function
    a = ':'.join(str(a) for a in args)
    return int(hashlib.sha256(a.encode('utf-8')).hexdigest(), 16)

def cryptrand(n=1024):
    return random.SystemRandom().getrandbits(n) % N

# A large safe prime (N = 2q+1, where q is prime)
# All arithmetic is done modulo N
# (generated using "openssl dhparam -text 1024")
N = simpleNum.safe_prime()
g = 2
k = H(N, g)  # Multiplier parameter (k=3 in legacy SRP-6)

print("#. H, N, g, and k are known beforehand to both client and server:")
global_print("H", "N", "g", "k")

print("0. server stores (I, s, v) in its password database")

# the server must first generate the password verifier
I = "niclas784"     # Username
p = "1626060998Da"   # Password
s = cryptrand(64)    # Salt for the user
x = H(s, I, p)       # Private key
v = pow(g, x, N)     # Password verifier
global_print("I", "p", "s", "x", "v")

print("1. client sends username I and public ephemeral value A to the server")
a = cryptrand()
A = pow(g, a, N)
global_print("I", "A")  # client->server (I, A)

print("2. server sends user's salt s and public ephemeral value B to client")
b = cryptrand()
B = (k * v + pow(g, b, N)) % N
global_print("s", "B")  # server->client (s, B)

print("3. client and server calculate the random scrambling parameter")
u = H(A, B)  # Random scrambling parameter
global_print("u")

print("4. client computes session key")
x = H(s, I, p)
S_c = pow(B - k * pow(g, x, N), a + u * x, N)
K_c = H(S_c)
global_print("S_c", "K_c")

print("5. server computes session key")
S_s = pow(A * pow(v, u, N), b, N)
K_s = H(S_s)
global_print("S_s", "K_s")

print("6. client sends proof of session key to server")
M_c = H(H(N) ^ H(g), H(I), s, A, B, K_c)
global_print("M_c")

print("7. server sends proof of session key to client")
M_s = H(A, M_c, K_s)
global_print("M_s")

#. H, N, g, and k are known beforehand to both client and server:
H = <function H at 0x10be93158>
N = 0x9fe176d723
g = 0x2
k = 0x3

0. server stores (I, s, v) in its password database
I = niclas784
p = 1626060998Da
s = 0x1696ef32e9
x = 0xa8a44f51ab35626005dd6dda12c49c774cba0c2eea2e4cede3c27b10f344a3c7
v = 0x1419a94d

1. client sends username I and public ephemeral value A to the server
I = niclas784
A = 0x77c0cfaece

2. server sends user's salt s and public ephemeral value B to client
s = 0x1696ef32e9
B = 0x97691eec

3. client and server calculate the random scrambling parameter
u = 0xaf6a61c5222b90cc2993f33cccc2c4ef0d97a598cc40d87051cd421f30f26ed7

4. client computes session key
S_c = 0x39fe7dc1e4
K_c = 0x7c68519626332f4f437e5a5cd41ec6bf43470b93e916aa56f6658141e7bbd24a

5. server computes session key
S_s = 0x39fe7dc1e4
K_s = 0x7c68519626332f4f437e5a5cd41ec6bf43470b93e916aa56f6658141e7bbd24a

6. client sends proof of session key to server
M_c = 0xc3fbcaf348be6100b7193af8a7eedc597dca5ba